In [11]:
# Load Libraries
import numpy as np
import os
import time # Sleeping
import pytz # Timezones
from twilio.rest import Client
# Getting .env information
from dotenv import load_dotenv

In [16]:
def send_texts(numbers, messages, account_sid, auth_token, twilio_number): # could refactor to send to user, that way we could inc. messages_sent in this function (better than havign to do it in parents)
    '''basic send function that takes in a list of numbers + list of messages and sends them out
    and returns a list of times that each message was sent
    '''
    client = Client(account_sid, auth_token)

    times = []
    for number, message in zip(numbers, messages):

        msg = client.messages.create(
        body= message,
        from_=twilio_number,
        to=number # replace with number in PROD
        ) # should check error handling, if needed based on SDK
        
        time.sleep(1) # Sleeping for 1 second between sending messages
        
        times.append(msg.date_updated.astimezone(pytz.timezone('America/Chicago'))) # Append times sent <- these come in utc

    return times

In [56]:
def check_unsubscriptions(numbers, account_sid, auth_token):
    '''Returns the indices of the phone numbers in numbers that have unsubscribed
    Which corresponds to record_ids_to_text'''


    unsubscribed_indices = []
    
    stop_key_words = ['STOP', 'STOPALL', 'UNSUBSCRIBE', 'CANCEL', 'END', 'QUIT']

    # Set up Twilio Client

    client = Client(account_sid, auth_token)

    # Iterate through the numbers 
    
    for i, number in enumerate(numbers):
        
        messages_from = client.messages.list(from_=number) # Check if the numbers have responded, messages_from is a list twilio objects
        # https://www.twilio.com/docs/messaging/api/message-resource?code-sample=code-list-all-message-resources&code-language=Python&code-sdk-version=8.x

        if len(messages_from) > 0: # If yes

            for message in messages_from: # What have they said?
    
                if message.body in stop_key_words: # If stopword

                    unsubscribed_indices += [i] # Add to unsubscriptions

                # else:

                #     message.delete() # We don't need to hold onto responses from people, do we? <- maybe save this for when we actually remove users

    return unsubscribed_indices

In [13]:
# Load .env info

load_dotenv()

account_sid = os.environ['TWILIO_ACCOUNT_SID']
auth_token = os.environ['TWILIO_AUTH_TOKEN']

twilio_number = os.environ['TWILIO_NUMBER']


In [14]:
# Set up test variables (new alert message)

numbers = [os.environ['LOCAL_PHONE']]

message = '''SPIKE ALERT!
Air quality is unhealthy in your area
https://map.purpleair.com/?select=145604/44.9723/-93.2447

Text STOP to unsubscribe
'''

messages = [message]

In [15]:
# call function

send_texts(numbers, messages, account_sid, auth_token, twilio_number)

[datetime.datetime(2023, 11, 10, 11, 7, 11, tzinfo=<DstTzInfo 'America/Chicago' CST-1 day, 18:00:00 STD>)]

In [6]:
# End alert Example

new_mess = '''Alert Over
Duration: 60 minutes 
Max value: 50.7 ug/m3

Report here - https://redcap.ahc.umn.edu/surveys/?s=LN3HHDCJXYCKFCLE&report_id=XXXXX-102623

'''
messages = [new_mess]

In [7]:
# call function

send_texts(numbers, messages, account_sid, auth_token, twilio_number)

[datetime.datetime(2023, 11, 8, 23, 38, 31, tzinfo=<UTC>)]